In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,SimpleRNN
import tensorflow as tf

In [3]:
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
#df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,1,0,31560.368,97906.322,30623.583,52909.261,42604.509,17521.203,42.950,44.145
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,1,0,19035.781,36973.722,30187.366,26585.677,41847.195,17577.073,42.927,44.209
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,1,0,120405.354,28886.583,96196.009,60325.969,45629.697,17632.943,42.904,44.273
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,1,0,8629.271,31655.750,35642.181,23684.576,48609.446,17688.812,42.881,44.337
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,1,0,31562.326,29065.500,8211.854,30046.873,49142.567,17744.682,42.858,44.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,1,0,40554.444,14324.708,37711.542,67747.671,49409.849,46077.069,10.607,79.307
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,1,0,31739.526,22046.601,65821.128,37511.137,46137.266,46106.208,10.566,79.354
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,1,0,15892.381,31801.521,21450.109,39168.139,45454.091,46135.347,10.526,79.401
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,1,0,22263.507,97828.229,41419.396,42477.179,39779.804,46164.486,10.485,79.448


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define evaluation function
def evaluate_forecast(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# --------------------------
# Function for each region
# --------------------------
def rbm_forecasting(region_code, display_name):
    print(f"\n📊 Processing region: {display_name}")

    # Filter the data
    region_df = df[df['Region'] == region_code].copy()
    region_df['Date'] = pd.to_datetime(region_df['Date'])
    region_df.set_index('Date', inplace=True)

    # Daily aggregation
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    # Scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    # Train/Test split
    split = int(0.8 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = y_scaled[:split], y_scaled[split:]
    test_dates = daily_data.index[split:]

    # Define RBM + NN pipeline
    rbm_nn = Pipeline(steps=[
        ('rbm', BernoulliRBM(n_components=64, learning_rate=0.06, n_iter=10, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42))
    ])

    # Train the model
    rbm_nn.fit(X_train, y_train)

    # Predict
    y_pred_scaled = rbm_nn.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    # ----------------------
    # Evaluation Metrics
    # ----------------------
    rmse_demand, mae_demand, mape_demand = evaluate_forecast(y_true[:, 0], y_pred[:, 0])
    rmse_co2, mae_co2, mape_co2 = evaluate_forecast(y_true[:, 1], y_pred[:, 1])

    print(f"{display_name} - Demand RMSE: {rmse_demand:.2f}, MAE: {mae_demand:.2f}, MAPE: {mape_demand:.2f}%")
    print(f"{display_name} - CO₂ RMSE: {rmse_co2:.2f}, MAE: {mae_co2:.2f}, MAPE: {mape_co2:.2f}%")

    result_df = pd.DataFrame({
        'Date': test_dates,
        'Region': display_name,
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
   # result_df.to_csv(f"rbm_nn_forecast_{region_code.lower()}.csv", index=False)

    # Return metrics
    return {
        'Region': display_name,
        'Demand': {'RMSE': rmse_demand, 'MAE': mae_demand, 'MAPE': mape_demand},
        'CO2': {'RMSE': rmse_co2, 'MAE': mae_co2, 'MAPE': mape_co2}
    }

# ✅ Run for both California and Texas
cal_metrics = rbm_forecasting("CAL", "California")
tex_metrics = rbm_forecasting("TEX", "Texas")


📊 Processing region: California
California - Demand RMSE: 2494.99, MAE: 1605.10, MAPE: 4.97%
California - CO₂ RMSE: 1113.51, MAE: 827.21, MAPE: 15.90%

📊 Processing region: Texas
Texas - Demand RMSE: 4875.72, MAE: 3659.13, MAPE: 7.15%
Texas - CO₂ RMSE: 3209.02, MAE: 2422.79, MAPE: 17.84%


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define evaluation function
def evaluate_forecast(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# --------------------------
# Function for each region
# --------------------------
def rbm_forecasting(region_code, display_name):
    print(f"\n📊 Processing region: {display_name}")

    # Filter the data
    region_df = df[df['Region'] == region_code].copy()
    region_df['Date'] = pd.to_datetime(region_df['Date'])
    region_df.set_index('Date', inplace=True)

    # Daily aggregation
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    # Scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    # Train/Test split
    split = int(0.75 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = y_scaled[:split], y_scaled[split:]
    test_dates = daily_data.index[split:]

    # Define RBM + NN pipeline
    rbm_nn = Pipeline(steps=[
        ('rbm', BernoulliRBM(n_components=64, learning_rate=0.06, n_iter=10, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42))
    ])

    # Train the model
    rbm_nn.fit(X_train, y_train)

    # Predict
    y_pred_scaled = rbm_nn.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    # ----------------------
    # Evaluation Metrics
    # ----------------------
    rmse_demand, mae_demand, mape_demand = evaluate_forecast(y_true[:, 0], y_pred[:, 0])
    rmse_co2, mae_co2, mape_co2 = evaluate_forecast(y_true[:, 1], y_pred[:, 1])

    print(f"{display_name} - Demand RMSE: {rmse_demand:.2f}, MAE: {mae_demand:.2f}, MAPE: {mape_demand:.2f}%")
    print(f"{display_name} - CO₂ RMSE: {rmse_co2:.2f}, MAE: {mae_co2:.2f}, MAPE: {mape_co2:.2f}%")


    result_df = pd.DataFrame({
        'Date': test_dates,
        'Region': display_name,
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
   # result_df.to_csv(f"rbm_nn_forecast_{region_code.lower()}.csv", index=False)

    # Return metrics
    return {
        'Region': display_name,
        'Demand': {'RMSE': rmse_demand, 'MAE': mae_demand, 'MAPE': mape_demand},
        'CO2': {'RMSE': rmse_co2, 'MAE': mae_co2, 'MAPE': mape_co2}
    }

# ✅ Run for both California and Texas
cal_metrics = rbm_forecasting("CAL", "California")
tex_metrics = rbm_forecasting("TEX", "Texas")


📊 Processing region: California
California - Demand RMSE: 2334.83, MAE: 1570.84, MAPE: 4.95%
California - CO₂ RMSE: 1117.26, MAE: 827.58, MAPE: 15.77%

📊 Processing region: Texas
Texas - Demand RMSE: 7479.84, MAE: 6280.16, MAPE: 12.15%
Texas - CO₂ RMSE: 2696.38, MAE: 2052.47, MAPE: 13.92%


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define evaluation function
def evaluate_forecast(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# --------------------------
# Function for each region
# --------------------------
def rbm_forecasting(region_code, display_name):
    print(f"\n📊 Processing region: {display_name}")

    # Filter the data
    region_df = df[df['Region'] == region_code].copy()
    region_df['Date'] = pd.to_datetime(region_df['Date'])
    region_df.set_index('Date', inplace=True)

    # Daily aggregation
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    # Scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    # Train/Test split
    split = int(0.7 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = y_scaled[:split], y_scaled[split:]
    test_dates = daily_data.index[split:]

    # Define RBM + NN pipeline
    rbm_nn = Pipeline(steps=[
        ('rbm', BernoulliRBM(n_components=64, learning_rate=0.06, n_iter=10, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42))
    ])

    # Train the model
    rbm_nn.fit(X_train, y_train)

    # Predict
    y_pred_scaled = rbm_nn.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    # ----------------------
    # Evaluation Metrics
    # ----------------------
    rmse_demand, mae_demand, mape_demand = evaluate_forecast(y_true[:, 0], y_pred[:, 0])
    rmse_co2, mae_co2, mape_co2 = evaluate_forecast(y_true[:, 1], y_pred[:, 1])

    print(f"{display_name} - Demand RMSE: {rmse_demand:.2f}, MAE: {mae_demand:.2f}, MAPE: {mape_demand:.2f}%")
    print(f"{display_name} - CO₂ RMSE: {rmse_co2:.2f}, MAE: {mae_co2:.2f}, MAPE: {mape_co2:.2f}%")

    result_df = pd.DataFrame({
        'Date': test_dates,
        'Region': display_name,
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
   # result_df.to_csv(f"rbm_nn_forecast_{region_code.lower()}.csv", index=False)

    # Return metrics
    return {
        'Region': display_name,
        'Demand': {'RMSE': rmse_demand, 'MAE': mae_demand, 'MAPE': mape_demand},
        'CO2': {'RMSE': rmse_co2, 'MAE': mae_co2, 'MAPE': mape_co2}
    }

# ✅ Run for both California and Texas
cal_metrics = rbm_forecasting("CAL", "California")
tex_metrics = rbm_forecasting("TEX", "Texas")


📊 Processing region: California
California - Demand RMSE: 2487.90, MAE: 1689.90, MAPE: 5.33%
California - CO₂ RMSE: 1080.12, MAE: 804.41, MAPE: 16.76%

📊 Processing region: Texas
Texas - Demand RMSE: 7132.35, MAE: 5936.98, MAPE: 11.69%
Texas - CO₂ RMSE: 2465.81, MAE: 1855.79, MAPE: 12.13%


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define evaluation function
def evaluate_forecast(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# --------------------------
# Function for each region
# --------------------------
def rbm_forecasting(region_code, display_name):
    print(f"\n📊 Processing region: {display_name}")

    # Filter the data
    region_df = df[df['Region'] == region_code].copy()
    region_df['Date'] = pd.to_datetime(region_df['Date'])
    region_df.set_index('Date', inplace=True)

    # Daily aggregation
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    # Scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    # Train/Test split
    split = int(0.65 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = y_scaled[:split], y_scaled[split:]
    test_dates = daily_data.index[split:]

    # Define RBM + NN pipeline
    rbm_nn = Pipeline(steps=[
        ('rbm', BernoulliRBM(n_components=64, learning_rate=0.06, n_iter=10, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42))
    ])

    # Train the model
    rbm_nn.fit(X_train, y_train)

    # Predict
    y_pred_scaled = rbm_nn.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    # ----------------------
    # Evaluation Metrics
    # ----------------------
    rmse_demand, mae_demand, mape_demand = evaluate_forecast(y_true[:, 0], y_pred[:, 0])
    rmse_co2, mae_co2, mape_co2 = evaluate_forecast(y_true[:, 1], y_pred[:, 1])

    print(f"{display_name} - Demand RMSE: {rmse_demand:.2f}, MAE: {mae_demand:.2f}, MAPE: {mape_demand:.2f}%")
    print(f"{display_name} - CO₂ RMSE: {rmse_co2:.2f}, MAE: {mae_co2:.2f}, MAPE: {mape_co2:.2f}%")

    # ----------------------
    # Plotting
    # ----------------------
    # plt.figure(figsize=(14, 5))
    # plt.plot(test_dates, y_true[:, 0], label='Actual Demand')
    # plt.plot(test_dates, y_pred[:, 0], label='Predicted Demand (RBM + NN)')
    # plt.title(f"{display_name} – Electricity Demand Forecast (RBM + NN)")
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    # plt.figure(figsize=(14, 5))
    # plt.plot(test_dates, y_true[:, 1], label='Actual CO₂ Emissions')
    # plt.plot(test_dates, y_pred[:, 1], label='Predicted CO₂ (RBM + NN)')
    # plt.title(f"{display_name} – CO₂ Emissions Forecast (RBM + NN)")
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    # Export Results to CSV
    result_df = pd.DataFrame({
        'Date': test_dates,
        'Region': display_name,
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
   # result_df.to_csv(f"rbm_nn_forecast_{region_code.lower()}.csv", index=False)

    # Return metrics
    return {
        'Region': display_name,
        'Demand': {'RMSE': rmse_demand, 'MAE': mae_demand, 'MAPE': mape_demand},
        'CO2': {'RMSE': rmse_co2, 'MAE': mae_co2, 'MAPE': mape_co2}
    }

# ✅ Run for both California and Texas
cal_metrics = rbm_forecasting("CAL", "California")
tex_metrics = rbm_forecasting("TEX", "Texas")


📊 Processing region: California
California - Demand RMSE: 2533.00, MAE: 1715.42, MAPE: 5.53%
California - CO₂ RMSE: 1077.85, MAE: 805.30, MAPE: 17.01%

📊 Processing region: Texas
Texas - Demand RMSE: 6525.97, MAE: 5422.93, MAPE: 10.93%
Texas - CO₂ RMSE: 2284.96, MAE: 1741.58, MAPE: 11.27%


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define evaluation function
def evaluate_forecast(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# --------------------------
# Function for each region
# --------------------------
def rbm_forecasting(region_code, display_name):
    print(f"\n📊 Processing region: {display_name}")

    # Filter the data
    region_df = df[df['Region'] == region_code].copy()
    region_df['Date'] = pd.to_datetime(region_df['Date'])
    region_df.set_index('Date', inplace=True)

    # Daily aggregation
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    # Scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    # Train/Test split
    split = int(0.6 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = y_scaled[:split], y_scaled[split:]
    test_dates = daily_data.index[split:]

    # Define RBM + NN pipeline
    rbm_nn = Pipeline(steps=[
        ('rbm', BernoulliRBM(n_components=64, learning_rate=0.06, n_iter=10, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(128, 64), activation='relu', max_iter=500, random_state=42))
    ])

    # Train the model
    rbm_nn.fit(X_train, y_train)

    # Predict
    y_pred_scaled = rbm_nn.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    # ----------------------
    # Evaluation Metrics
    # ----------------------
    rmse_demand, mae_demand, mape_demand = evaluate_forecast(y_true[:, 0], y_pred[:, 0])
    rmse_co2, mae_co2, mape_co2 = evaluate_forecast(y_true[:, 1], y_pred[:, 1])

    print(f"{display_name} - Demand RMSE: {rmse_demand:.2f}, MAE: {mae_demand:.2f}, MAPE: {mape_demand:.2f}%")
    print(f"{display_name} - CO₂ RMSE: {rmse_co2:.2f}, MAE: {mae_co2:.2f}, MAPE: {mape_co2:.2f}%")

    result_df = pd.DataFrame({
        'Date': test_dates,
        'Region': display_name,
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
   # result_df.to_csv(f"rbm_nn_forecast_{region_code.lower()}.csv", index=False)

    # Return metrics
    return {
        'Region': display_name,
        'Demand': {'RMSE': rmse_demand, 'MAE': mae_demand, 'MAPE': mape_demand},
        'CO2': {'RMSE': rmse_co2, 'MAE': mae_co2, 'MAPE': mape_co2}
    }

# ✅ Run for both California and Texas
cal_metrics = rbm_forecasting("CAL", "California")
tex_metrics = rbm_forecasting("TEX", "Texas")


📊 Processing region: California
California - Demand RMSE: 2388.70, MAE: 1645.90, MAPE: 5.18%
California - CO₂ RMSE: 1171.79, MAE: 882.22, MAPE: 17.49%

📊 Processing region: Texas
Texas - Demand RMSE: 6022.78, MAE: 4705.56, MAPE: 9.28%
Texas - CO₂ RMSE: 2494.46, MAE: 1924.64, MAPE: 12.57%
